## Copy fields using GetAccept *API*

Functions to copy fields from a source template to a destination template. First, create the new template ("destination template") manually in the GetAccept platform. Then add roles to the new template, manually as well, according to the roles in the source template. IMPORTANT: the role name will be used as a key when mapping roles (and its corresponding fields) between the templates, so please make sure the role names are spelled exactly the same in source and destination templates.

In [ ]:
# import dependencies
import requests
import os
import io
from google.colab import files

In [ ]:
#@title Enter GetAccept information (ALL fields are required)
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
BASEURL = "https://api.getaccept.com/v1" #@param {type:"string"}
SOURCE_ENTITY = "" #@param {type: "string"}
DESTINATION_ENTITY = "" #@param {type: "string"}
#@markdown Find template id by going to the document in GetAccept \
#@markdown and copy the id from the URL in the browser, eg. abc123de
SOURCE_TEMPLATE_ID = "" #@param {type: "string"}
DESTINATION_TEMPLATE_ID = "" #@param {type: "string"}

In [ ]:
# Make auth request and store auth header
if EMAIL=="" or PASSWORD=="" or BASEURL=="" or SOURCE_ENTITY=="" or DESTINATION_ENTITY=="" or SOURCE_TEMPLATE_ID=="" or DESTINATION_TEMPLATE_ID=="":
  raise TypeError("Please fill out all fields!")
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": SOURCE_ENTITY }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))


In [ ]:
# If destination is another entity, also auth this
if DESTINATION_ENTITY == SOURCE_ENTITY:
  destination_auth_headers = source_auth_headers
else:
  response = requests.get(BASEURL+"/refresh/"+DESTINATION_ENTITY, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    destination_auth_headers = { "Authorization": "bearer " + data["access_token"]}
    print("Authenticated to destination entity " + DESTINATION_ENTITY)
  else:
    raise TypeError(data["errors"], "Please check your DESTINATION_ENTITY parameter")

In [ ]:
# Get roles from templates
response = requests.get(BASEURL+"/templates/"+SOURCE_TEMPLATE_ID+"/roles",headers=source_auth_headers)
source_roles_data = response.json()
response = requests.get(BASEURL+"/templates/"+DESTINATION_TEMPLATE_ID+"/roles",headers=destination_auth_headers)
dest_roles_data = response.json()
source_roles_dict = {}
dest_roles_dict = {}
for role in source_roles_data["roles"]:
  if role["role_name"] == "":
    source_roles_dict[role["role_id"]] = role["email"]
  else:
    source_roles_dict[role["role_id"]] = role["role_name"]
for role in dest_roles_data["roles"]:
  if role["role_name"] == "":
    dest_roles_dict[role["email"]] = role["role_id"]
  else:
    dest_roles_dict[role["role_name"]] = role["role_id"]
print("Source roles: " + str(source_roles_dict))
print("Dest roles: " + str(dest_roles_dict))

In [ ]:
# Get fields for source template
response = requests.get(BASEURL+"/documents/"+SOURCE_TEMPLATE_ID+"/fields",headers=source_auth_headers)
field_data = response.json()
print("Fields: " + ", ".join(map(lambda x:x["field_type"]+": "+x["field_value"]+"  "+x["field_label"], field_data["fields"])))

In [ ]:
# Get original document pages
response = requests.get(BASEURL+"/documents/"+SOURCE_TEMPLATE_ID+"/pages", headers=source_auth_headers)
page_data = response.json()
document_page_num = {}
for page in page_data["pages"]:
  document_page_num[page["page_id"]] = page["page_num"]

print("pages: " + ", ".join(map(lambda x : str(x["page_num"]) + ": " + x["page_id"], page_data["pages"])))
# print(document_page_num)
  

In [ ]:
# Copy original fields to the new document
field_copy_counter = 0
for field in field_data["fields"]:
  payload = field.copy()
  payload["page_num"] = document_page_num[payload["page_id"]]
  for e in ["page_id","field_id"]: 
    payload.pop(e, None)
  if payload["field_options"]:
    payload["field_options"] = ','.join(map(lambda x : x["value"], payload["field_options"]))
  if payload["recipient_id"] != 0:
    print(payload["recipient_id"])
    print(dest_roles_dict[source_roles_dict[payload["recipient_id"]]])
    payload["recipient_id"] = dest_roles_dict[source_roles_dict[payload["recipient_id"]]]
  response = requests.post(BASEURL+"/documents/"+DESTINATION_TEMPLATE_ID+"/fields", json=payload, headers=destination_auth_headers)
  new_field_data = response.json()
  if "field_id" in new_field_data:
    field_copy_counter = field_copy_counter + 1
  else:
    raise TypeError(new_field_data, "Something went wrong")

print("Copied " + str(field_copy_counter) + " fields to the document/template")